# Preliminary Analysis Data ADNI

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tabulate import tabulate

In [24]:
#Load the different datasets to analyse the data, rename columns if necessary 
data_raw = pd.read_csv('/analysis/ritter/data/ADNI/ADNI_BIDS_skull_stripped/participants.csv', delimiter = ',')
participant_data = pd.read_csv('/analysis/ritter/projects/AD/Budding_Spectral_Analysis/data/ROSTER.csv', delimiter = ',')
participant_data = participant_data.rename(columns = {'PTID':'SubjectID'})
conversion_data = pd.read_csv('/analysis/ritter/projects/AD/Budding_Spectral_Analysis/data/DXSUM_PDXCONV_ADNIALL.csv', delimiter = ',')
adas_data = pd.read_csv('/analysis/ritter/projects/AD/Budding_Spectral_Analysis/data/ADASSCORES.csv', delimiter = ',')
adas_data = adas_data.rename(columns = {'VISCODE':'session'})

failed_preprocessing = pd.read_csv('/analysis/ritter/projects/AD/Budding_Spectral_Analysis/data/failed_preprocessing.csv', delimiter = ',')

In [25]:
#Add ADAS data to the general data 

#Rename adas data timepoints
adas_data = adas_data.replace({'bl':'screen', 'm06':'month06', 'm12':'month12', 'm18':'month18','m24':'month24'})

#Get subject IDs that are in our data set
subject_ID_adas_dupl = pd.merge(participant_data[['RID', 'SubjectID']], adas_data)
subject_ID_adas = subject_ID_adas_dupl.drop_duplicates()
subject_ID_adas = subject_ID_adas[subject_ID_adas['SubjectID'].isin(data_raw['SubjectID'])]

#Merge adni data with original data
full_data_adas = pd.merge(data_raw, subject_ID_adas[['SubjectID', 'session', 'TOTAL11', 'TOTALMOD']])
full_data_adas = full_data_adas.drop_duplicates()
data_raw = full_data_adas

## Check for conversion and reversion

In [26]:
#Select only the ADNI1 data of the conversion table
conversion_ADNI1 = conversion_data[conversion_data['Phase'] == 'ADNI1']

#Merge participant ID data with conversion table to get subject IDs and remove duplicates
subject_ID_conversion_dupl = pd.merge(participant_data[['RID', 'SubjectID']], conversion_ADNI1)
print(subject_ID_conversion['SubjectID'].unique().shape)

#Select the conversion IDs of subjects in the data set we will use 
subject_ID_conversion = subject_ID_conversion[subject_ID_conversion['SubjectID'].isin(data_raw['SubjectID'])]
print(subject_ID_conversion.shape)
print(data_raw.shape)
print(subject_ID_conversion['SubjectID'].isin(data_raw['SubjectID']).all())

(818,)
(3831, 54)
(2770, 20)
True


In [27]:
# Merge data with conversion data, drop duplicates
full_data = pd.merge(data_raw, subject_ID_conversion[['SubjectID', 'RID', 'DXCONV']], on = 'SubjectID', how = "outer")
full_data = full_data.drop_duplicates()
print(full_data['SubjectID'].unique().shape)
print(np.sum(full_data['DXCONV']))
print(full_data.shape)
# export_full_data = full_data.to_csv(r'/analysis/ritter/projects/AD/Budding_Spectral_Analysis/data/' + 'full_data.csv', index = None, header = True)

#Select participants that have conversion 
converters_all_times = full_data[full_data['DXCONV'] != 0.0]
data_without_converters = data_raw[~data_raw['SubjectID'].isin(converters_all_times['SubjectID'])]
data = data_without_converters

#Print relevant information
print('Excluded scans', len(converters_all_times['SubjectID']))
print('#Participants without converters/reverters:', len(data_without_converters['SubjectID'].unique()))
print('#Participants raw data:', len(data_raw['SubjectID'].unique()))
print('Excluded participants total:', len(converters_all_times['SubjectID'].unique()))
print(len(data))
assert converters_all_times['SubjectID'].isin(data_raw['SubjectID']).all()

(811,)
926.0
(3614, 22)
Excluded scans 844
#Participants without converters/reverters: 602
#Participants raw data: 811
Excluded participants total: 209
1948


In [32]:
# Make a csv of the converter data 
data_csv = converters_all_times
export_data_csv = data_csv.to_csv(r'/analysis/ritter/projects/AD/Budding_Spectral_Analysis/data/' + 'data_converters_old.csv', index = None, header = True)

## Exclude scans with failed preprocessing

In [6]:
# Exclude the failed preprocessing scans 
data_failures = pd.merge(data, failed_preprocessing, on = ['SubjectID', 'session'], how = "inner")
data = data[~data['Image ID'].isin(data_failures['Image ID'])]
print(len(data_failures), 'have been excluded due to failed preprocessing. Still', len(data), 'scans in the datset.')

9 have been excluded due to failed preprocessing. Still 1959 scans in the datset.


## Subjects per group and timepoint

In [7]:
#Total number of scans per group and excluded scans per group for in dataframe
excluded_data = [[len(data_raw[data_raw['Research Group'] == 'AD']), 
                  len(converters_all_times[converters_all_times['Research Group'] == 'AD']),
                 len(converters_all_times[converters_all_times['Research Group'] == 'AD']['SubjectID'].unique())],
                 [len(data_raw[data_raw['Research Group'] == 'MCI']), 
                  len(converters_all_times[converters_all_times['Research Group'] == 'MCI']),
                 len(converters_all_times[converters_all_times['Research Group'] == 'MCI']['SubjectID'].unique())],
                 [len(data_raw[data_raw['Research Group'] == 'CN']), 
                  len(converters_all_times[converters_all_times['Research Group'] == 'CN']),
                 len(converters_all_times[converters_all_times['Research Group'] == 'CN']['SubjectID'].unique())], 
                [len(data_raw), len(converters_all_times['SubjectID']),
                len(converters_all_times['SubjectID'].unique())]]

In [8]:
#Dataframe with number of excluded scans
excluded_scans = pd.DataFrame(excluded_data, columns = ['Total Scans', 'Excluded Scans', 'Excluded Participants'], 
                              index = ['AD', 'MCI', 'CN', 'Total'])
excluded_scans

,Total Scans,Excluded Scans,Excluded Participants
AD,552,13,3
MCI,1470,776,190
CN,782,69,19
Total,2804,858,212


In [9]:
#Masks groups
is_ad = data['Research Group']=='AD'
is_mci = data['Research Group']=='MCI'
is_cn = data['Research Group']=='CN'

In [10]:
#Data per group 
data_AD = data[is_ad]
data_MCI = data[is_mci]
data_CN = data[is_cn]

In [11]:
#Get number of scans per subject group 
number_scans = [[len(data_AD), len(pd.unique(data_AD['SubjectID']))], 
                [len(data_MCI), len(pd.unique(data_MCI['SubjectID']))],
                [len(data_CN), len(pd.unique(data_CN['SubjectID']))],
                [len(data), len(pd.unique(data['SubjectID']))]]

scans_per_group = pd.DataFrame(number_scans, columns=['Total', 'Unique patients'], 
                               index = ['AD', 'MCI', 'CN', 'Total'])
scans_per_group

,Total,Unique patients
AD,539,185
MCI,710,211
CN,710,210
Total,1959,606


In [12]:
#Data per timepoint and group AD
data_AD_screen = data_AD[data_AD['session'] == 'screen']
data_AD_month06 = data_AD[data_AD['session'] == 'month06']
data_AD_month12 = data_AD[data_AD['session'] == 'month12']
data_AD_month18 = data_AD[data_AD['session'] == 'month18']
data_AD_month24 = data_AD[data_AD['session'] == 'month24']

In [13]:
#Data per timepoint and group MCI
data_MCI_screen = data_MCI[data_MCI['session'] == 'screen']
data_MCI_month06 = data_MCI[data_MCI['session'] == 'month06']
data_MCI_month12 = data_MCI[data_MCI['session'] == 'month12']
data_MCI_month18 = data_MCI[data_MCI['session'] == 'month18']
data_MCI_month24 = data_MCI[data_MCI['session'] == 'month24']

In [14]:
#Data per timepoint and group CN 
data_CN_screen = data_CN[data_CN['session'] == 'screen']
data_CN_month06 = data_CN[data_CN['session'] == 'month06']
data_CN_month12 = data_CN[data_CN['session'] == 'month12']
data_CN_month18 = data_CN[data_CN['session'] == 'month18']
data_CN_month24 = data_CN[data_CN['session'] == 'month24']

In [15]:
#Get list for dataframe
data_pd = [[len(data_AD_screen), len(data_MCI_screen), len(data_CN_screen)],
        [len(data_AD_month06), len(data_MCI_month06), len(data_CN_month06)],
        [len(data_AD_month12), len(data_MCI_month12), len(data_CN_month12)],
        [len(data_AD_month18), len(data_MCI_month18), len(data_CN_month18)],
        [len(data_AD_month24), len(data_MCI_month24), len(data_CN_month24)]]

In [16]:
#Create dataframe with number of participants in each group 
participant_data = pd.DataFrame(data_pd, columns = ['AD', 'MCI', 'CN'], 
                                index = ['screen', 'month06', 'month12', 'month18', 'month24'])

participant_data

,AD,MCI,CN
screen,181,208,206
month06,129,147,177
month12,131,154,175
month18,0,95,0
month24,98,106,152


## Mean and standard deviation of cognitive tests

In [17]:
#Get the mean and standard deviation of a variable for a certain subject group
def get_mean_sd(subject_group, column):
    mean = subject_group[column].mean()
    sd = subject_group[column].std()
    return mean, sd

#Define a list of all subject groups
data_all = [data_AD_screen, data_MCI_screen, data_CN_screen,
        data_AD_month06, data_MCI_month06, data_CN_month06,
        data_AD_month12, data_MCI_month12, data_CN_month12,
        data_AD_month18, data_MCI_month18, data_CN_month18,
        data_AD_month24, data_MCI_month24, data_CN_month24]

In [18]:
#Get a list with mean and standard deviation for all subject groups 
def get_table_mean(data_all, column):
    test = [get_mean_sd(data, column) for data in data_all]
    test = np.reshape(test, (5,6))
    return test

In [19]:
#Define header and indices for table
header = pd.MultiIndex.from_product([['AD', 'MCI', 'CN'], ['mean', 'sd']])
rows = ['screen', 'month06', 'month12', 'month18', 'month24']

#Create tables with scores for all the tests 
mmse = pd.DataFrame(get_table_mean(data_all, 'MMSE Total Score'), 
                    columns = header, index = rows)
gdscale = pd.DataFrame(get_table_mean(data_all, 'GDSCALE Total Score'), columns = header, index = rows)
npi_q = pd.DataFrame(get_table_mean(data_all, 'NPI-Q Total Score'), columns = header, index = rows)
cdr = pd.DataFrame(get_table_mean(data_all, 'Global CDR'), columns = header, index = rows)
faq = pd.DataFrame(get_table_mean(data_all, 'FAQ Total Score'), columns = header, index = rows)
adas11 = pd.DataFrame(get_table_mean(data_all, 'TOTAL11'), columns = header, index = rows)
adas13 = pd.DataFrame(get_table_mean(data_all, 'TOTALMOD'), columns = header, index = rows)

In [20]:
# Print tables with scores 
print('MMSE', mmse, '\n')
print('GDSCALE', gdscale, '\n')
print('NPI-Q', npi_q, '\n')
print('CDR', cdr,'\n')
print('FAQ', faq, '\n')
print('ADAS11', adas11, '\n')
print('ADAS13', adas13, '\n')

MMSE                 AD                  MCI                   CN          
              mean        sd       mean        sd       mean        sd
screen   23.226519  2.049001  27.278846  1.790780  29.111650  1.022746
month06  22.472868  3.089998  27.129252  2.436234  29.096045  0.969323
month12  20.715385  4.578742  27.296053  2.693723  29.155172  1.189575
month18        NaN       NaN  27.010526  2.958920        NaN       NaN
month24  18.552083  5.879258  27.211538  3.249117  29.211921  1.017246 

GDSCALE                AD                 MCI                  CN          
             mean        sd      mean        sd      mean        sd
screen   1.657459  1.431321  1.615385  1.399156  0.800971  1.101750
month06       NaN       NaN       NaN       NaN       NaN       NaN
month12  1.811024  1.871814  1.743421  1.806892  0.942529  1.266339
month18       NaN       NaN       NaN       NaN       NaN       NaN
month24  2.010638  2.091944  1.865385  1.790252  1.105960  1.545541 

NPI-Q     

## Division in training and test data

In [21]:
#Determine for which participants a scan is available at all four timepoints
def scans_all_timepoints(data1, data2, data3, data4, column):
    intersection1 = pd.merge(data1[column], data2[column], how = 'inner')
    intersection2 = pd.merge(data3[column], intersection1, how = 'inner')
    participants_all_timepoints = pd.merge(data4[column], intersection2, how = 'inner')
    return participants_all_timepoints

In [22]:
#Get number of participants with all timepoints for different groups 
all_timepoints_AD = scans_all_timepoints(data_AD_screen, data_AD_month06, 
                                         data_AD_month12, data_AD_month24, 'SubjectID')
all_timepoints_MCI = scans_all_timepoints(data_MCI_screen, data_MCI_month06, 
                                         data_MCI_month12, data_MCI_month24, 'SubjectID')
all_timepoints_CN = scans_all_timepoints(data_CN_screen, data_CN_month06, 
                                         data_CN_month12, data_CN_month24, 'SubjectID')

print('#scans all timepoints AD:', len(all_timepoints_AD))
print('#scans all timepoints MCI:', len(all_timepoints_MCI))
print('#scans all timepoints CN:', len(all_timepoints_CN))

#scans all timepoints AD: 92
#scans all timepoints MCI: 98
#scans all timepoints CN: 144


In [23]:
#Determine number of test points for all groups (20% of total)
test_points_AD = int(0.2* len(all_timepoints_AD))
# test_points_MCI = int(0.2 * len(all_timepoints_MCI))
test_points_CN = int(0.2 * len(all_timepoints_CN))
print('Number of test scans for AD: {}, CN: {}'.format(test_points_AD, test_points_CN))

Number of test scans for AD: 18, CN: 28


In [24]:
#Set random seed 
random_state = 43

#Randomly select a sample from the subjects
sample_AD = all_timepoints_AD.sample(test_points_AD, random_state = random_state)
# sample_MCI = all_timepoints_MCI.sample(test_points_MCI)
sample_CN = all_timepoints_CN.sample(test_points_CN, random_state = random_state)

In [25]:
# print(sample['Subject ID'])
selected_AD = data_AD['SubjectID'].isin(sample_AD['SubjectID'])
data_AD_holdout = data_AD[selected_AD]
data_AD_train = data_AD[~selected_AD]

# All MCI data will be used to test on so not necessary to select this
# selected_MCI = data_MCI['Subject ID'].isin(sample_MCI['Subject ID'])
# data_MCI_test = data_MCI[selected_MCI]
# data_MCI_train = data_MCI[~selected_MCI]

selected_CN = data_CN['SubjectID'].isin(sample_CN['SubjectID'])
data_CN_holdout = data_CN[selected_CN]
data_CN_train = data_CN[~selected_CN]

In [26]:
# Get list of subjects for AD and CN (training sets)
subjects_AD_train = data_AD_train['SubjectID'].unique()
subjects_CN_train = data_CN_train['SubjectID'].unique()

val_subjects_AD = int(0.15 * len(subjects_AD_train))
val_subjects_CN = int(0.15 * len(subjects_CN_train))
print(val_subjects_AD, val_subjects_CN)

# Get validation set for AD and CN 
subjects_AD_train, subjects_AD_val = train_test_split(subjects_AD_train, 
                                                      test_size=val_subjects_AD, random_state=random_state)
subjects_CN_train, subjects_CN_val = train_test_split(subjects_CN_train, 
                                                      test_size=val_subjects_CN, random_state=random_state)

subjects_train = np.concatenate([subjects_AD_train, subjects_CN_train])
subjects_val = np.concatenate([subjects_AD_val, subjects_CN_val])

25 27


In [27]:
# Make dataframe with all scans for the testing set (and testing set + MCI)
data_training = data[data.apply(lambda row: row['SubjectID'] in subjects_train, axis=1)]
data_val = data[data.apply(lambda row: row['SubjectID'] in subjects_val, axis=1)]
data_holdout = pd.concat([data_AD_holdout, data_CN_holdout], ignore_index = True)
# data_holdout_all = data[~data['SubjectID'].isin(data_training['SubjectID'])]

In [28]:
def print_all_stats(df, df_train, df_test, df_val):
    headers = ['Images', '-> AD', '-> MCI', '-> CN', 'Patients', '-> AD', '-> MCI', '-> CN']
    
    def get_stats(df):
        df_ad = df[df['Research Group'] == 'AD']
        df_mci = df[df['Research Group'] == 'MCI']
        df_cn = df[df['Research Group'] == 'CN']
        return [len(df), len(df_ad), len(df_mci), len(df_cn), len(df['SubjectID'].unique()), 
                len(df_ad['SubjectID'].unique()), len(df_mci['SubjectID'].unique()), len(df_cn['SubjectID'].unique())]
        
    stats = []
    stats.append(['ALL'] + get_stats(df))
    stats.append(['Train'] + get_stats(df_train))
    stats.append(['Holdout'] + get_stats(df_test))
    stats.append(['Val'] + get_stats(df_val))
        
    print(tabulate(stats, headers = headers))
    print()

In [29]:
print_all_stats(data, data_training, data_holdout, data_val)

           Images    -> AD    -> MCI    -> CN    Patients    -> AD    -> MCI    -> CN
-------  --------  -------  --------  -------  ----------  -------  --------  -------
ALL          1959      539       710      710         606      185       211      210
Train         901      390         0      511         297      142         0      155
Holdout       184       72         0      112          46       18         0       28
Val           164       77         0       87          52       25         0       27



In [30]:
# Comment out when you want to create a new csv file for the newly selected participants
# export_csv_training = data_training.to_csv(r'/analysis/ritter/projects/AD/Budding_Spectral_Analysis/data/' + 'ADNI_training.csv', index = None, header = True)
# export_csv_holdout = data_holdout.to_csv(r'/analysis/ritter/projects/AD/Budding_Spectral_Analysis/data/' + 'ADNI_holdout.csv', index = None, header = True)
# export_csv_validation = data_val.to_csv(r'/analysis/ritter/projects/AD/Budding_Spectral_Analysis/data/' + 'ADNI_validation.csv', index = None, header = True)
# export_csv_testing_all = data_testing_all.to_csv(r'/analysis/ritter/projects/AD/Budding_Spectral_Analysis/data/' + 'ADNI_testing_all.csv', index = None, header = True)